In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 20.2 MB/s eta 0:00:00


In [2]:
pip install webdriver_manager

##Working fine for now


In [4]:
import os
import requests
import selenium
from bs4 import BeautifulSoup
import re
import csv
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import datetime



# Function to extract publish date from URL
def extract_publish_date(url, consider_future_years=True):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        meta_date = soup.find('meta', {'property': 'article:published_time'}) or \
                    soup.find('meta', {'name': 'date'}) or \
                    soup.find('meta', {'name': 'publish_date'}) or \
                    soup.find('meta', {'itemprop': 'datePublished'})

        if meta_date and meta_date.get('content'):
            publish_date = meta_date['content']
            return check_future_year(publish_date, consider_future_years)

        text = soup.get_text()
        date_match = re.search(r'\b(20[2-9][0-9])\b', text)
        if date_match:
            return check_future_year(date_match.group(1), consider_future_years)

    except requests.exceptions.HTTPError as e:
        print(f"HTTPError for {url}: {e}")
    except Exception as e:
        print(f"Error fetching publish date from {url}: {e}")
    return None

def check_future_year(publish_date, consider_future_years):
    current_year = datetime.datetime.now().year
    publish_year = int(publish_date.split('-')[0])

    if not consider_future_years and publish_year > current_year:
        print(f"Skipping future year: {publish_year}")
        return None
    return publish_date

def find_latest_report(urls, consider_future_years=True):
    latest_date = None
    latest_url = None
    report_data = []
    report_names = ['quality of life' ,'Global Terrorism Index','World Competitveness Yearbook', 'Global Talent Competitiveness Index', 'ICT Development Index', 'Open Data inventory Index',
                    'Sustainable Development Goals Index', 'Social progress Index', 'Human Development Index', 'Gender Inequality Index',# 'world report',
                    'Competittive Industrial Performance Index', 'World happiness Report','Human Capital index', 'Logistic Performance Index', 'Rule Of Law index', 'Women Business and Law',
                    ]

    for index, url in enumerate(urls):
        print(f"Checking publish dates on: {url}")
        publish_date = extract_publish_date(url, consider_future_years)
        status = "Not Published"

        if publish_date:
            print(f"Found publish date: {publish_date}")
            if not latest_date or publish_date > latest_date:
                latest_date = publish_date
                latest_url = url

        report_data.append([url, report_names[index], publish_date, status])

    if latest_date:
        for report in report_data:
            if report[0] == latest_url:
                report[3] = "Published"
        result = f"The latest report was published on {latest_date}. URL: {latest_url}"
    else:
        result = "No publish dates found on the provided URLs."
    print(result)
    return result, report_data

def save_to_csv(report_data):
    file_path = 'report_data.csv'
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["URL", "Report Name", "Extracted Date", "Publication Status"])
        writer.writerows(report_data)
    print(f"CSV file saved to {file_path}")

def send_email(sender_email, sender_password, recipient_email, subject, body):
    try:
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = ", ".join(recipient_email)#recipient_email
        message['Subject'] = subject
        message.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, message.as_string())  # Send to multiple emails

        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

if __name__ == "__main__":
    urls = [
        'https://www.economicsandpeace.org/global-terrorism-index/',
        'https://www.numbeo.com/quality-of-life/',

        'https://www.imd.org/centers/wcc/world-competitiveness-center/rankings/world-digital-competitiveness-ranking/',
        'https://www.insead.edu/global-talent-competitiveness-index',
        'https://www.itu.int/en/ITU-D/Statistics/Pages/IDI/default.aspx',
        'https://odin.opendatawatch.com/',
        'https://sdgs.un.org/',
        'https://www.socialprogress.org/',
        'https://hdr.undp.org/',
        'https://hdr.undp.org/data/gender-inequality-index',
       # 'https://www.hrw.org/world-report/2025',
        'https://www.unido.org',
        'https://worldhappiness.report/',
        'https://databank.worldbank.org/source/human-capital-index',
        'https://lpi.worldbank.org/',
        'https://worldjusticeproject.org/rule-of-law-index/',
        'https://wbl.worldbank.org/',


    ]
    result, report_data = find_latest_report(urls, consider_future_years=False)
    save_to_csv(report_data)
    sender_email = "nishi.fcsc@gmail.com"
    sender_password = "pirn tfaz lfcn ioss"
    recipient_email = ["nishipk001@gmail.com", "nishi.kochunni@fcsc.gov.ae"]
    subject = "Latest Report Publish Date"
    body = result
    send_email(sender_email, sender_password, recipient_email, subject, body)


Checking publish dates on: https://www.economicsandpeace.org/global-terrorism-index/
Found publish date: 2025
Checking publish dates on: https://www.numbeo.com/quality-of-life/
Found publish date: 2025
Checking publish dates on: https://www.imd.org/centers/wcc/world-competitiveness-center/rankings/world-digital-competitiveness-ranking/
Found publish date: 2024-11-13T22:55:34+00:00
Checking publish dates on: https://www.insead.edu/global-talent-competitiveness-index
Found publish date: 2023
Checking publish dates on: https://www.itu.int/en/ITU-D/Statistics/Pages/IDI/default.aspx
Found publish date: 2022
Checking publish dates on: https://odin.opendatawatch.com/
Found publish date: 2022
Checking publish dates on: https://sdgs.un.org/
Skipping future year: 2030
Checking publish dates on: https://www.socialprogress.org/
Found publish date: 2025
Checking publish dates on: https://hdr.undp.org/
Found publish date: 2023
Checking publish dates on: https://hdr.undp.org/data/gender-inequality-in

Multiple Email

In [ ]:
import os
import requests
import selenium
from bs4 import BeautifulSoup
import re
import csv
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import datetime
from google.colab import drive

drive.mount('/content/drive')

# Function to extract publish date from URL
def extract_publish_date(url, consider_future_years=True):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        meta_date = soup.find('meta', {'property': 'article:published_time'}) or \
                    soup.find('meta', {'name': 'date'}) or \
                    soup.find('meta', {'name': 'publish_date'}) or \
                    soup.find('meta', {'itemprop': 'datePublished'})

        if meta_date and meta_date.get('content'):
            publish_date = meta_date['content']
            return check_future_year(publish_date, consider_future_years)

        text = soup.get_text()
        date_match = re.search(r'\b(20[2-9][0-9])\b', text)
        if date_match:
            return check_future_year(date_match.group(1), consider_future_years)

    except requests.exceptions.HTTPError as e:
        print(f"HTTPError for {url}: {e}")
    except Exception as e:
        print(f"Error fetching publish date from {url}: {e}")
    return None

def check_future_year(publish_date, consider_future_years):
    current_year = datetime.datetime.now().year
    publish_year = int(publish_date.split('-')[0])

    if not consider_future_years and publish_year > current_year:
        print(f"Skipping future year: {publish_year}")
        return None
    return publish_date

def find_latest_report(urls, consider_future_years=True):
    latest_date = None
    latest_url = None
    report_data = []
    report_names = ['quality of life' ,'Global Terrorism Index','World Competitveness Yearbook', 'Global Talent Competitiveness Index', 'ICT Development Index', 'Open Data inventory Index',
                    'Sustainable Development Goals Index', 'Social progress Index', 'Human Development Index', 'Gender Inequality Index',# 'world report',
                    'Competittive Industrial Performance Index', 'World happiness Report','Human Capital index', 'Logistic Performance Index', 'Rule Of Law index', 'Women Business and Law',
                    ]

    for index, url in enumerate(urls):
        print(f"Checking publish dates on: {url}")
        publish_date = extract_publish_date(url, consider_future_years)
        status = "Not Published"

        if publish_date:
            print(f"Found publish date: {publish_date}")
            if not latest_date or publish_date > latest_date:
                latest_date = publish_date
                latest_url = url

        report_data.append([url, report_names[index], publish_date, status])

    if latest_date:
        for report in report_data:
            if report[0] == latest_url:
                report[3] = "Published"
        result = f"The latest report was published on {latest_date}. URL: {latest_url}"
    else:
        result = "No publish dates found on the provided URLs."
    print(result)
    return result, report_data

def save_to_csv(report_data):
    file_path = '/content/drive/My Drive/report_data.csv'
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["URL", "Report Name", "Extracted Date", "Publication Status"])
        writer.writerows(report_data)
    print(f"CSV file saved to {file_path}")

def send_email(sender_email, sender_password, recipient_email, subject, body):
    try:
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = ", ".join(recipient_email)#recipient_email
        message['Subject'] = subject
        message.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, message.as_string())  # Send to multiple emails

        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

if __name__ == "__main__":
    urls = [
        'https://www.economicsandpeace.org/global-terrorism-index/',
        'https://www.numbeo.com/quality-of-life/',

        'https://www.imd.org/centers/wcc/world-competitiveness-center/rankings/world-digital-competitiveness-ranking/',
        'https://www.insead.edu/global-talent-competitiveness-index',
        'https://www.itu.int/en/ITU-D/Statistics/Pages/IDI/default.aspx',
        'https://odin.opendatawatch.com/',
        'https://sdgs.un.org/',
        'https://www.socialprogress.org/',
        'https://hdr.undp.org/',
        'https://hdr.undp.org/data/gender-inequality-index',
       # 'https://www.hrw.org/world-report/2025',
        'https://www.unido.org',
        'https://worldhappiness.report/',
        'https://databank.worldbank.org/source/human-capital-index',
        'https://lpi.worldbank.org/',
        'https://worldjusticeproject.org/rule-of-law-index/',
        'https://wbl.worldbank.org/',


    ]
    result, report_data = find_latest_report(urls, consider_future_years=False)
    save_to_csv(report_data)
    sender_email = "nishi.fcsc@gmail.com"
    sender_password = "pirn tfaz lfcn ioss"
    recipient_email = ["nishipk001@gmail.com", "nishi.kochunni@fcsc.gov.ae"]
    subject = "Latest Report Publish Date"
    body = result
    send_email(sender_email, sender_password, recipient_email, subject, body)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checking publish dates on: https://www.economicsandpeace.org/global-terrorism-index/
Found publish date: 2025
Checking publish dates on: https://www.numbeo.com/quality-of-life/
Found publish date: 2025
Checking publish dates on: https://www.imd.org/centers/wcc/world-competitiveness-center/rankings/world-digital-competitiveness-ranking/
Found publish date: 2024-11-13T22:55:34+00:00
Checking publish dates on: https://www.insead.edu/global-talent-competitiveness-index
Found publish date: 2023
Checking publish dates on: https://www.itu.int/en/ITU-D/Statistics/Pages/IDI/default.aspx
Found publish date: 2022
Checking publish dates on: https://odin.opendatawatch.com/
Found publish date: 2022
Checking publish dates on: https://sdgs.un.org/
Skipping future year: 2030
Checking publish dates on: https://www.socialprogress.org/
Found publish date: 2025
Checking publish d